In [ ]:
# 需先建立好video和result兩個空資料夾，再開始執行

In [1]:
import cv2
import time, os
import numpy as np
import matplotlib.pyplot as plt
from skimage.feature import greycomatrix, greycoprops
import scipy.special as special

In [2]:
def beta_correction( f, a = 2.0, b = 2.0 ):
    x = np.linspace( 0, 1, 256 )
    y = special.betainc( a, b, x ) * 255
    g = f.copy()
    nr, nc = f.shape[:2]
    for i in range(nr):
        for j in range(nc):
            g[i,j] = np.uint8(y[f[i,j]])
    return g

In [1]:
#Capture the video
def get_images_from_video(video_name, time_F):
    video_images = []
    vc = cv2.VideoCapture(video_name)
    c = 1
    
    if vc.isOpened(): 
        rval, video_frame = vc.read()
    else:
        rval = False

    while rval:
        rval, video_frame = vc.read()
        
        if(c % time_F == 0):
            video_images.append(video_frame)     
        c = c + 1
    vc.release()
    
    return video_images

time_F = 5 #決定每幾張圖片要抓取一次
video_name = 'texture_video.avi' 
video_images = get_images_from_video(video_name, time_F)

#將圖片數字前方補 0，使圖片能按照順序排列
for i in range(0, len(video_images)):
    if i < 10:
        num = "000" + str(i+1)
    elif i < 100:
        num = "00" + str(i+1)
    elif i < 1000:
        num = "0" + str(i+1)
    else:
        num = str(i+1)
    cv2.imwrite( "./video/img"+num+".jpg", video_images[i])

In [5]:
#可調整迴圈範圍，決定要偵測的數量
for k in range(300,360):
    start = time.time()
    if k < 10:
        num = "000" + str(k+1)
    elif k < 100:
        num = "00" + str(k+1)
    elif k < 1000:
        num = "0" + str(k+1)
    else:
        num = str(k+1)

    image = cv2.imread('./video/img'+str(num)+'.jpg')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

############################################################
# 以下為瑕疵偵測部分

    #beta處理
    image = beta_correction(image, a = 2.0, b = 2.0 )

    size_filter = 25
    offset = 5

    nr,nc = image.shape[:2]
    x_filter = int((nr)/offset)
    y_filter = int((nc)/offset)

    end_r = nr - size_filter + offset
    end_c = nc - size_filter + offset

    co_contrast = np.zeros((x_filter ,y_filter))
    co_correlation = np.zeros((x_filter ,y_filter))
    co_dissimilarity = np.zeros((x_filter,y_filter))

    for i in range(0, end_r, offset):
        for j in range(0, end_c, offset):
            f = image[ i : i + size_filter , j : j + size_filter ]
            result = greycomatrix( f, [1],[0] )
            co_contrast[int(i/offset),int(j/offset)] = float(greycoprops(result, 'contrast'))
            co_correlation[int(i/offset),int(j/offset)] = float(greycoprops(result, 'correlation'))
            co_dissimilarity[int(i/offset),int(j/offset)] = float(greycoprops(result,'dissimilarity'))

    #contrast
    contrast_mean = round(np.mean(co_contrast))
    contrast_std = round(np.std(co_contrast))
    contrast_max = round(np.max(co_contrast))
    #dissimilarity
    dissimilarity_mean = round(np.mean(co_dissimilarity))
    dissimilarity_std = round(np.std(co_dissimilarity))
    #correlation
    correlation_mean = np.mean(co_correlation)
    correlation_std = np.std(co_correlation)

    #門檻值設定
    contrast_mean_threshold = 125        #circle大於 125.033578 
    contrast_std_threshold = 39         #circle大於 38.544783 
    contrast_max_threshold = 246         #circle大於 246.250000 
    dissimilarity_mean_threshold = 8.46   #circle大於 8.459510 
    dissimilarity_std_threshold = 2.3    #circle大於 2.244871 
    circle_count = 0
    is_circle = True

    #過關，判斷線性or圓形
    if contrast_mean > contrast_mean_threshold:circle_count+=1
    if contrast_std > contrast_std_threshold:circle_count+=1
    if contrast_max > contrast_max_threshold:circle_count+=1
    if dissimilarity_mean > dissimilarity_mean_threshold:circle_count+=1
    if dissimilarity_std > dissimilarity_std_threshold:circle_count+=1
    if circle_count >= 3:
#         print('image is circle')
        pass
    else:
#         print('image is linear')
        is_circle = False
#     print(f'correlation_mean = {correlation_mean}\n correlation_std = {correlation_std}')

    #建立空白的偵測影像
    detection = np.full((nr, nc), 255,dtype='uint8')

    #決定影像的門檻值
    if is_circle:
        threshold = 0.822
        size = 15
    else:
        size = 10
        if correlation_std > 0.15:
            threshold = correlation_mean + (correlation_std*0.75)
        else:
            threshold = correlation_mean + correlation_std
#     print('threshold = ',threshold) 

    #將瑕疵點標記出來
    for i in range(x_filter):
        for j in range(y_filter):
            if co_correlation[i,j] > threshold:
                for m in range(j*offset,j*offset+size):
                    for n in range(i*offset,i*offset+size):
                        try:
                            detection[n][m] = 0
                        except:
                            pass

# 以上為瑕疵偵測部分
############################################################

    cv2.imwrite( "./result/"+str(num)+".png", detection)
    end = time.time()
    print(num, end-start)

0151 66.32979369163513
0152 66.36258625984192
0153 64.64932417869568
0154 84.64396905899048
0155 56.66156721115112
0156 71.89095044136047
0157 71.38208293914795
0158 70.46803045272827
0159 60.04459810256958
0160 70.45729231834412


In [6]:
#image to video
path = "./result/"
filelist = os.listdir(path)
fps = 2 
size = (800, 600) 

#使用 XVID 編碼
video = cv2.VideoWriter("VideoTest.avi", 
        cv2.VideoWriter_fourcc('X', 'V', 'I', 'D'), fps, size)

for item in filelist:
    item = path + item
    img = cv2.imread(item)
    video.write(img)

video.release()
cv2.destroyAllWindows()